## Generating QR images for the model training


In [15]:
import os
import qrcode
import cv2
import numpy as np
from zxing import *
from keras.preprocessing import image
import tensorflow as tf

# Crea un directorio para guardar los códigos QR
if not os.path.exists('qr_codes'):
    os.makedirs('qr_codes')

# Genera 1000 códigos QR aleatorios y los guarda en el directorio
for i in range(1000):
    data = 'QR code ' + str(i)
    filename = 'qr_codes/qr_code_' + str(i) + '.png'
    qr = qrcode.QRCode(version=1, box_size=10, border=4)
    qr.add_data(data)
    qr.make(fit=True)
    img = qr.make_image(fill_color="black", back_color="white")
    img.save(filename)


## Generate model parameters

In [9]:
# Carga las imágenes de los códigos QR
X = []
for i in range(1000):
    filename = 'qr_codes/qr_code_' + str(i) + '.png'
    img = cv2.imread(filename, cv2.IMREAD_GRAYSCALE)  # Carga la imagen en escala de grises
    img = cv2.resize(img, (100, 100))  # Redimensiona la imagen

    # Agrega la imagen preprocesada a la lista de características
    X.append(img)

In [10]:
# Convierte la lista de características en un ndarray
X = np.array(X)

# Carga las etiquetas de los códigos QR
y = []
for i in range(1000):
    data = i
    y.append(data)

# Convierte la lista de etiquetas en un ndarray
y = np.array(y)

# Agrega una dimensión adicional para el número de canales
X = np.expand_dims(X, axis=-1)

In [11]:
## Building the CNN model

In [12]:
# Agrega una dimensión adicional para el número de canales
X = np.expand_dims(X, axis=-1)

# Construye el modelo CNN
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(100, 100, 1)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1000)
])

# Compila el modelo
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

# Entrena el modelo con los datos preprocesados
model.fit(X, y, epochs=10, batch_size=32)


# Guarda el modelo en un archivo HDF5
model.save('qr-challenge-with-ml/qr_model.keras')

Epoch 1/10
32/32 [==============================] - 2s 68ms/step - loss: 15.4108 - accuracy: 0.0000e+00
Epoch 2/10
32/32 [==============================] - 2s 68ms/step - loss: 6.9098 - accuracy: 0.0000e+00
Epoch 3/10
32/32 [==============================] - 2s 68ms/step - loss: 6.9092 - accuracy: 0.0000e+00
Epoch 4/10
32/32 [==============================] - 2s 69ms/step - loss: 6.9101 - accuracy: 0.0020
Epoch 5/10
32/32 [==============================] - 2s 69ms/step - loss: 6.8998 - accuracy: 0.0010
Epoch 6/10
32/32 [==============================] - 2s 69ms/step - loss: 6.8319 - accuracy: 0.0030
Epoch 7/10
32/32 [==============================] - 2s 68ms/step - loss: 6.9440 - accuracy: 0.0040
Epoch 8/10
32/32 [==============================] - 2s 68ms/step - loss: 6.9076 - accuracy: 0.0000e+00
Epoch 9/10
32/32 [==============================] - 2s 68ms/step - loss: 6.8985 - accuracy: 0.0030
Epoch 10/10
32/32 [==============================] - 2s 68ms/step - loss: 6.9074 - accuracy:

In [13]:
## Make a prediction

In [16]:
# Crea un diccionario que asocia cada etiqueta numérica con un nombre de objeto
label_map = {i: 'QR code ' + str(i) for i in range(1000)}

# Carga una imagen con un código QR
img = cv2.imread('qr_code_test.png')

# Guarda la imagen en un archivo temporal
cv2.imwrite('temp_img.png', img)

# Crea un objeto de detector de códigos QR
detector = BarCodeReader()

# Detecta los códigos QR en la imagen
barcode = detector.decode('temp_img.png')

# Convierte el objeto BarCode a una lista
if barcode is not None:
    barcode_list = list(vars(barcode).values())
    print('QR code:', barcode_list)

    # Decodifica el contenido de los códigos QR encontrados
    x, y = barcode_list[5][0]
    w, h = barcode_list[5][2]
    roi = img[int(y):int(y+h), int(x):int(x+w)]
    roi = cv2.resize(roi, (100, 100))
    roi = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)  # Convierte a escala de grises
    roi = np.expand_dims(roi, axis=0)
    roi = np.expand_dims(roi, axis=-1)  # Agrega una dimensión para el número de canales

    # Haga una predicción utilizando el modelo
    prediction = model.predict(roi)
    predicted_class = np.argmax(prediction, axis=-1)[0]  # Índice de la clase predicha
    predicted_label = label_map[predicted_class]  # Etiqueta correspondiente a la clase predicha
    print('Predicción:', predicted_label)

# Borra el archivo temporal
os.remove('temp_img.png')

QR code: ['QR code 1001', 'QR code 1001', 'file:///Users/toglia/Documents/qr-challenge-with-ml/temp_img.png', 'QR_CODE', 'TEXT', [(75.0, 215.0), (75.0, 75.0), (215.0, 75.0)]]
1/1 [==============================] - 0s 56ms/step
Predicción: QR code 549
